In [1]:
import os
import sys
import numpy as np
import ipympl
import matplotlib
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.io import fits
from astropy.table import Table

from mmtwfs.wfs import *
from mmtwfs.zernike import ZernikeVector
from mmtwfs.telescope import MMT

from camsrv.header import HEADER_MAP

from pathlib import Path

In [42]:
def get_header_info(keylist, fitsfile):
    info = []
    h = fits.open(fitsfile)
    hdr = h[0].header
    for k in keylist:
        if k in hdr:
            info.append(hdr[k])
        else:
            print(k)
            info.append(None)
    h.close()
    return info

# model coefficients are taken from latest settings.xml on hacksaw
def f5_model(key, el, t_oss):
    model = {}
    sin_el = np.sin(el * u.deg)
    cos_el = np.cos(el * u.deg)
    model['TRANSX'] = 787.7 - 507.6 * sin_el - 947.2 * cos_el + 0.0 * t_oss
    model['TRANSY'] = 1780.0 - 135.8 * sin_el + 1538.0 * cos_el + 0.0 * t_oss
    model['TRANSZ'] = 13608.5 + 671.1 * sin_el - 286.7 * cos_el - 13.1 * t_oss
    model['TILTX'] = 236.7 - 66.1 * sin_el + 119.6 * cos_el + 0.0 * t_oss
    model['TILTY'] = 70.5 + 20.5 * sin_el + 72.0 * cos_el + 0.0 * t_oss
    return model[key]

def bino_model(key, el, t_oss):
    model = {}
    sin_el = np.sin(el * u.deg)
    cos_el = np.cos(el * u.deg)
    model['TRANSX'] = 787.7 - 507.6 * sin_el - 947.2 * cos_el + 0.0 * t_oss
    model['TRANSY'] = 1780.0 - 135.8 * sin_el + 1538.0 * cos_el + 0.0 * t_oss
    model['TRANSZ'] = 13608.5 + 671.1 * sin_el - 286.7 * cos_el - 13.1 * t_oss
    model['TILTX'] = 236.7 - 66.1 * sin_el + 119.6 * cos_el + 0.0 * t_oss
    model['TILTY'] = 70.5 + 20.5 * sin_el + 72.0 * cos_el + 0.0 * t_oss
    return model[key]

def f9_model(key, el, t_oss):
    model = {}
    sin_el = np.sin(el * u.deg)
    cos_el = np.cos(el * u.deg)
    model['TRANSX'] = 2097.8 - 628.2 * sin_el - 1147.6 * cos_el + 0.0 * t_oss
    model['TRANSY'] = -919.7 + 199.7 * sin_el + 1046.9 * cos_el + 0.0 * t_oss
    model['TRANSZ'] = 293.4 + 725.2 * sin_el + 81.3 * cos_el - 46.7 * t_oss
    model['TILTX'] = 78.5 - 45.8 * sin_el + 48.3 * cos_el + 0.0 * t_oss
    model['TILTY'] = -83.3 + 56.6 * sin_el + 157.4 * cos_el + 0.0 * t_oss
    return model[key]

M2_KEYS = ['TRANSX', 'TILTY', 'TRANSY', 'TILTX', 'TRANSZ']

# plot the M2 position data in three panels. up/down motion is comprised of a combination of TRANSY and TILTX while
# left/right motion is comprised of TRANSX and -TILTY. for zero-coma pointing shifts, the ratio of trans/tilt is 9.45 um/arcsec
# for F/5 and 5.86 um/arcsec for F/9.

def plot_m2(t, xaxis='EL', model=None):
    x = t[xaxis]
    fig, axes = plt.subplots(3, sharex=True, figsize=(6, 9))
    pred = {}
    for k in M2_KEYS:
        if model is not None:
            pred[k] = model(k, t['EL'], t['OSSTEMP'])
        else:
            pred[k] = 0.0
    for a in axes:
        a.xaxis.grid()
    axes[0].scatter(x, t['TRANSZ']-pred['TRANSZ'], color='r')
    axes[0].set_ylabel("Focus (um)")
    axes[1].scatter(x, t['TRANSY']-pred['TRANSY'], color='r')
    axes[1].set_ylabel("Y Translation (um)", color='r')
    ax1_twin = axes[1].twinx()
    ax1_twin.scatter(x, t['TILTX']-pred['TILTX'], color='b')
    ax1_twin.set_ylabel("X Tilt (arcsec)", color='b', rotation=270, labelpad=20)
    axes[2].scatter(x, t['TRANSX']-pred['TRANSX'], color='r')
    axes[2].set_ylabel("X Translation (um)", color='r')
    axes[2].set_xlim(20, 90)
    axes[2].set_xlabel("Elevation (deg)")
    ax2_twin = axes[2].twinx()
    ax2_twin.scatter(x, t['TILTY']-pred['TILTY'], color='b')
    ax2_twin.set_ylabel("Y Tilt (arcsec)", color='b', rotation=270, labelpad=20)
    fig.tight_layout()
    fig.subplots_adjust(hspace=0)
    plt.show()

In [3]:
%cd /Volumes/Seagate2TB/wfsdat

/Volumes/Seagate2TB/wfsdat


In [4]:
with open("bino_2018.txt") as f:
    bino_files = f.readlines()
with open("f9wfs_2018.txt") as f:
    f9_files = f.readlines()

In [5]:
keys = []
for k in HEADER_MAP:
    keys.append(HEADER_MAP[k]['fitskey'])

Check the wavefront fits and only use the data where the total wavefront RMS is less than 1000 nm. A stricter cutoff of 500 nm was investigated. However, it didn't affect the results much for the Binospec data and cut out a lot of the F/9 data.

In [53]:
rows = []
for f in f9_files:
    p = Path(f.strip())
    zv = ZernikeVector()
    zv.load(str(p))
    if zv.rms < 1000.0 * u.nm:
        fitsfile = str(p.parent) + "/" + p.stem
        row = [fitsfile]
        hinfo = get_header_info(keys, fitsfile)
        row += hinfo
        rows.append(row)

f9_table = Table(rows=rows, names=["filename"] + keys)

In [56]:
rows = []
for f in bino_files:
    p = Path(f.strip())
    zv = ZernikeVector()
    zv.load(str(p))
    if zv.rms < 1000.0 * u.nm:
        fitsfile = str(p.parent) + "/" + p.stem
        row = [fitsfile]
        try:
            hinfo = get_header_info(keys, fitsfile)
            row += hinfo
            rows.append(row)
        except:
            pass

bino_table = Table(rows=rows, names=["filename"] + keys)
len(bino_table), len(f9_table)

In [57]:
plot_m2(bino_table, model=bino_model)

FigureCanvasNbAgg()

In [58]:
plot_m2(f9_table, model=f9_model)

FigureCanvasNbAgg()

In [45]:
bino_resid = {}
for k in M2_KEYS:
    bino_resid[k] = bino_table[k] - bino_model(k, bino_table['EL'], bino_table['OSSTEMP'])
f9_resid = {}
for k in M2_KEYS:
    f9_resid[k] = f9_table[k] - f9_model(k, f9_table['EL'], f9_table['OSSTEMP'])

In [47]:
az = bino_table['AZ']
az[az < 0] += 360.
plt.scatter(az, bino_resid['TILTY'])
plt.show()

FigureCanvasNbAgg()

In [48]:
f9_az = f9_table['AZ']
f9_az[f9_az < 0] += 360.
plt.scatter(f9_az, f9_resid['TILTY'])
plt.show()

FigureCanvasNbAgg()

In [22]:
plt.scatter(az, tilty_resid)
plt.show()

FigureCanvasNbAgg()

In [26]:
plt.scatter(bino_table['MJD'], az)
plt.show()

FigureCanvasNbAgg()